# Spark: Assignment Group F


**Table of Contents:** 

1. PySpark Environment Setup
2. Dataframe Setup 
3. Metadata Analysis
4. Data Cleaning
5. Business questions
6. Food color Analysis 


## 1. PySpark Environment Setup

In [38]:
#importing necessary libraries to make the environment run

import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import numpy as np

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## 2. Dataframe setup

In [39]:
DF1 = spark.read \
                 .option("inferSchema", "true") \
                 .option("header","true") \
                 .option("delimiter", "\t") \
                 .csv("en.openfoodfacts.org.products.US.with_headers.csv")

## 3. Metadata analysis
### 3.1 Display schema and size of the DataFrame

In [3]:
from IPython.display import display, Markdown

DF1.printSchema()

display(Markdown("This DataFrame has **%d rows**." % DF1.count()))

print((DF1.count(),len(DF1.columns)))

root
 |-- code: double (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: integer (nullable = true)
 |-- created_datetime: timestamp (nullable = true)
 |-- last_modified_t: integer (nullable = true)
 |-- last_modified_datetime: timestamp (nullable = true)
 |-- product_name: string (nullable = true)
 |-- abbreviated_product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- packaging_text: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- categories_tags: string (nullable = true)
 |-- categories_en: string (nullable = true)
 |-- origins: string (nullable = true)
 |-- origins_tags: string (nullable = true)
 |-- origins_en: string (nullable = true)
 |-- manufacturing_places: str

This DataFrame has **346468 rows**.

(346468, 184)


### 3.2 Get one sample from the data set

In [40]:
DF1.cache() 
DF1.sample(False, 0.1).take(1)

[Row(code=18630.0, url='http://world-en.openfoodfacts.org/product/0000000018630/organic-coconut-chips-grizzlies', creator='usda-ndb-import', created_t=1489055722, created_datetime=datetime.datetime(2017, 3, 9, 11, 35, 22), last_modified_t=1545997202, last_modified_datetime=datetime.datetime(2018, 12, 28, 12, 40, 2), product_name='Organic Coconut Chips', abbreviated_product_name=None, generic_name=None, quantity=None, packaging=None, packaging_tags=None, packaging_text=None, brands='Grizzlies', brands_tags='grizzlies', categories=None, categories_tags=None, categories_en=None, origins=None, origins_tags=None, origins_en=None, manufacturing_places=None, manufacturing_places_tags=None, labels='Organic', labels_tags='en:organic', labels_en='Organic', emb_codes=None, emb_codes_tags=None, first_packaging_code_geo=None, cities=None, cities_tags=None, purchase_places=None, stores=None, countries='United States', countries_tags='en:united-states', countries_en='United States', ingredients_text=

In [5]:
DF1.schema.names

['code',
 'url',
 'creator',
 'created_t',
 'created_datetime',
 'last_modified_t',
 'last_modified_datetime',
 'product_name',
 'abbreviated_product_name',
 'generic_name',
 'quantity',
 'packaging',
 'packaging_tags',
 'packaging_text',
 'brands',
 'brands_tags',
 'categories',
 'categories_tags',
 'categories_en',
 'origins',
 'origins_tags',
 'origins_en',
 'manufacturing_places',
 'manufacturing_places_tags',
 'labels',
 'labels_tags',
 'labels_en',
 'emb_codes',
 'emb_codes_tags',
 'first_packaging_code_geo',
 'cities',
 'cities_tags',
 'purchase_places',
 'stores',
 'countries',
 'countries_tags',
 'countries_en',
 'ingredients_text',
 'allergens',
 'allergens_en',
 'traces',
 'traces_tags',
 'traces_en',
 'serving_size',
 'serving_quantity',
 'no_nutriments',
 'additives_n',
 'additives',
 'additives_tags',
 'additives_en',
 'ingredients_from_palm_oil_n',
 'ingredients_from_palm_oil',
 'ingredients_from_palm_oil_tags',
 'ingredients_that_may_be_from_palm_oil_n',
 'ingredients_t

In [6]:
#doing a summary analysis of all the column, before cleaning

print ("Summary of columns:")
DF1.summary().toPandas()

Summary of columns:


,summary,code,url,creator,created_t,last_modified_t,product_name,abbreviated_product_name,generic_name,quantity,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,count,346468,346468,346468,346468,346468,340995,4,3991,11871,...,11,238980,0,0,0,3,1701,0,2,0
1,mean,2.439143437932444E50,None,Infinity,1.537011007757152E9,1.5829966209006488E9,6.63058998982E10,None,1671.6666666666667,38.271707746478874,...,361.0863636363636,9.136852456272491,None,None,None,0.07986666666666666,1.3610176366843035E-4,None,8.0138,None
2,stddev,1.435717205340317E53,None,NaN,5.0365319903437294E7,2.3907735249800734E7,1.8445598471286032E11,None,2887.617414640197,143.54644165405333,...,281.14807138846703,9.093587745080477,None,None,None,0.06721944163211513,5.760899037653907E-4,None,11.294192351824012,None
3,min,1.0,http://world-en.openfoodfacts.org/product/0000...,11vjspdpt,1332933734,1421079327,"!, Chocolate Bear Crunch Popcorn",Ferrero rocher t42,'guilt free',(1lb 4.6oz) 584g,...,14.8,-14,None,None,None,0.016,0.0,None,0.0276,None
4,25%,4.00002345E10,None,Infinity,1489074211,1587583908,180.0,None,4.0,1.0,...,166.5,1,None,None,None,0.016,8.3E-6,None,0.0276,None
5,50%,7.3140024311E10,None,Infinity,1489144192,1587635759,479.0,None,5.0,1.0,...,323.38,10,None,None,None,0.0736,2.31E-5,None,0.0276,None
6,75%,7.17544137368E11,None,Infinity,1587660476,1587664958,7.2392348312E10,None,5006.0,8.0,...,480.26,16,None,None,None,0.15,9.41E-5,None,16.0,None
7,max,8.450851484235123E55,http://world-en.openfoodfacts.org/product/9999...,zoneblockscommunity,1614042745,1614043140,울무차,Rigatoni 500g imu eu,蒜蓉,مقدار,...,1002.4,36,None,None,None,0.15,0.0171429,None,16.0,None


## 4. Data cleaning

### 4.1 Data entities, metrics and dimensions

* **Entities:** Product
* **Metrics:** energy-kcal_100g, fat_100g, saturated-fat_100g, carbohydrates_100g, sugars_100g, proteins_100g, salt_100g and many more columns with information regarding 100g content
* **Dimensions:** created_t, created_datetime, last_modified_t, last_modified_datetime, url

### 4.2 Column categorization

**General information**: Creator, Created_datetime, Created_t, Last_modified_datetime, Last_modified_t, Product_name, Quantity<br>
**Tags**: Packaging_tags, Countries_en, Categories_en<br>
**Ingredients**: Traces_en, Ingredients_text<br>
**Misc.data**: Main_category_en, Image_url, Additives_en, Additives_tags<br>
**Nutrition facts**: Energy-kcal_100g, Fat_100g, Saturated-fat_100g, Sugars_100g, Sodium_100g, Salt_100g<br>

In [7]:
#making a new dataframe called DF2 with the columns of interest given by the professor

from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit, avg

DF2 = DF1.select("Creator",\
                 "Created_datetime",\
                 "Created_t",\
                 "Last_modified_datetime",\
                 "Last_modified_t",\
                 "Product_name",\
                 "Countries_en",\
                 "Traces_en",\
                 "Additives_tags",\
                 "Main_category_en",\
                 "Image_url",\
                 "Quantity",\
                 "Packaging_tags",\
                 "Categories_en",\
                 "Ingredients_text",\
                 "Additives_en",\
                 "Energy-kcal_100g",\
                 "Fat_100g",\
                 "Saturated-fat_100g",\
                 "Sugars_100g",\
                 "Salt_100g",\
                 "Sodium_100g")



In [8]:
#here we are taking a sample of the new dataframe DF2

DF2.cache()
DF2.sample(False, 0.1).take(2)

[Row(Creator='usda-ndb-import', Created_datetime=datetime.datetime(2017, 3, 9, 11, 35, 31), Created_t=1489055731, Last_modified_datetime=datetime.datetime(2018, 12, 28, 12, 36, 48), Last_modified_t=1545997008, Product_name='Organic Salted Nut Mix', Countries_en='United States', Traces_en=None, Additives_tags=None, Main_category_en=None, Image_url=None, Quantity=None, Packaging_tags=None, Categories_en=None, Ingredients_text='Organic hazelnuts, organic cashews, organic walnuts almonds, organic sunflower oil, sea salt.', Additives_en=None, Energy-kcal_100g=607.0, Fat_100g=57.14, Saturated-fat_100g=5.36, Sugars_100g=3.57, Salt_100g=1.22428, Sodium_100g=0.489712),
 Row(Creator='usda-ndb-import', Created_datetime=datetime.datetime(2017, 3, 9, 11, 35, 30), Created_t=1489055730, Last_modified_datetime=datetime.datetime(2018, 12, 28, 12, 36, 48), Last_modified_t=1545997008, Product_name='Organic Long Grain White Rice', Countries_en='United States', Traces_en=None, Additives_tags=None, Main_cat

### 4.3 Column profiling

#### General information:

In [9]:
#summarization of General information related columns

print ("Summary of General information:")
DF2.select("Creator","Created_t","Created_datetime","Last_modified_t",\
           "Last_modified_datetime","Product_name","Quantity").summary().show()


#checking null values of General information related columns

print("Checking for nulls on General information:")
DF2.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Creator","Created_t","Created_datetime",\
                                                               "Last_modified_t","Last_modified_datetime",\
                                                               "Product_name","Quantity"]]).show()


#checking distinct values of General information related columns

print("Checking amount of distinct values in General information:")
DF2.select([countDistinct(c).alias(c) for c in ["Creator","Created_t","Created_datetime",\
                                                "Last_modified_t","Last_modified_datetime",\
                                                "Product_name","Quantity"]]).show()


Summary of General information:
+-------+-------------------+--------------------+--------------------+--------------------+------------------+
|summary|            Creator|           Created_t|     Last_modified_t|        Product_name|          Quantity|
+-------+-------------------+--------------------+--------------------+--------------------+------------------+
|  count|             346468|              346468|              346468|              340995|             11871|
|   mean|           Infinity| 1.537011007757152E9|1.5829966209006488E9|    6.63058998982E10|38.271707746478874|
| stddev|                NaN|5.0365319903437294E7|2.3907735249800734E7|1.844559847128603...|143.54644165405333|
|    min|          11vjspdpt|          1332933734|          1421079327|!, Chocolate Bear...|  (1lb 4.6oz) 584g|
|    25%|           Infinity|          1489074211|          1587583908|               180.0|               1.0|
|    50%|           Infinity|          1489144192|          1587635759| 

#### Tags:

In [10]:
#summarization of Tags related columns

print ("Summary of Tags:")
DF2.select("Countries_en","Categories_en","Packaging_tags").summary().show()


#checking null values of Tags related columns

print("Checking for nulls on Tags:")
DF2.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Countries_en","Categories_en",\
                                                               "Packaging_tags"]]).show()


#checking distinct values of Tags related columns

print("Checking amount of distinct values in Tags:")
DF2.select([countDistinct(c).alias(c) for c in ["Countries_en","Categories_en",\
                                                "Packaging_tags"]]).show()


Summary of Tags:
+-------+--------------------+------------------+--------------------+
|summary|        Countries_en|     Categories_en|      Packaging_tags|
+-------+--------------------+------------------+--------------------+
|  count|              346468|            292653|                6819|
|   mean|                null|139.04545454545453|  144.29166666666666|
| stddev|                null|115.98459555874825|  485.61001343148547|
|    min|Albania,Algeria,A...|                 0|04-pe-ld,21-pap,g...|
|    25%|                null|              50.0|                 1.0|
|    50%|                null|             120.0|                 4.0|
|    75%|                null|             230.0|                30.0|
|    max|Virgin Islands of...|              شيبس|                  包|
+-------+--------------------+------------------+--------------------+

Checking for nulls on Tags:
+------------+-------------+--------------+
|Countries_en|Categories_en|Packaging_tags|
+------------+-

#### Ingredients:

In [11]:
#summarization of Ingredients related columns

print ("Summary of Ingredients:")
DF2.select("Traces_en","Ingredients_text").summary().show()


#checking null values of Ingredients related columns

print("Checking for nulls on Ingredients:")
DF2.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Traces_en","Ingredients_text"]]).show()


#checking distinct values of Ingredients related columns

print("Checking amount of distinct values in Ingredients:")
DF2.select([countDistinct(c).alias(c) for c in ["Traces_en","Ingredients_text"]]).show()

Summary of Ingredients:
+-------+--------------------+---------------------------------+
|summary|           Traces_en|                 Ingredients_text|
+-------+--------------------+---------------------------------+
|  count|                3825|                           321703|
|   mean|                null|                              2.0|
| stddev|                null|                              NaN|
|    min|1-milkfat,Vitamin...|             !ngredients: porc...|
|    25%|                null|                              2.0|
|    50%|                null|                              2.0|
|    75%|                null|                              2.0|
|    max|              ro:apa|주의문구·부정·불량식품 신고는 ...|
+-------+--------------------+---------------------------------+

Checking for nulls on Ingredients:
+---------+----------------+
|Traces_en|Ingredients_text|
+---------+----------------+
|   342643|           24765|
+---------+----------------+

Checking amount of distinct

#### Misc.data:

In [12]:
#summarization of Misc.data related columns

print ("Summary of Misc.data:")
DF2.select("Main_category_en","Additives_tags","Additives_en","Image_url").summary().show()


#checking null values of Misc.data related columns

print("Checking for nulls on Misc.data:")
DF2.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Main_category_en","Additives_tags","Additives_en","Image_url"]]).show()


#checking distinct values of Misc.data related columns

print("Checking amount of distinct values in Misc.data:")
DF2.select([countDistinct(c).alias(c) for c in ["Main_category_en","Additives_tags","Additives_en","Image_url"]]).show()

Summary of Misc.data:
+-------+------------------+--------------+--------------------+--------------------+
|summary|  Main_category_en|Additives_tags|        Additives_en|           Image_url|
+-------+------------------+--------------+--------------------+--------------------+
|  count|            292653|        209009|              209009|               35619|
|   mean|135.63636363636363|          null|                null|                null|
| stddev|101.34742787246076|          null|                null|                null|
|    min|                 0|       en:e100|     E100 - Curcumin|https://static.op...|
|    25%|              70.0|          null|                null|                null|
|    50%|             120.0|          null|                null|                null|
|    75%|             190.0|          null|                null|                null|
|    max|            调味品|       en:e999|E999 - Quillaia e...|https://static.op...|
+-------+------------------+-------

#### Nutrition facts:

In [13]:
#summarization of Nutrition facts related columns

print ("Summary of Nutrition facts:")
DF2.select("Energy-kcal_100g","Fat_100g","Saturated-fat_100g","Sugars_100g","Salt_100g","Sodium_100g").summary().show()


#checking null values of Nutrition facts related columns

print("Checking for nulls on Nutrition facts:")
DF2.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Energy-kcal_100g","Fat_100g","Saturated-fat_100g","Sugars_100g","Salt_100g","Sodium_100g"]]).show()


#checking distinct values of Nutrition facts related columns

print("Checking amount of distinct values in Nutrition facts:")
DF2.select([countDistinct(c).alias(c) for c in ["Energy-kcal_100g","Fat_100g","Saturated-fat_100g","Sugars_100g","Salt_100g","Sodium_100g"]]).show()

Summary of Nutrition facts:
+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|  Energy-kcal_100g|          Fat_100g|Saturated-fat_100g|       Sugars_100g|         Salt_100g|       Sodium_100g|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|            328717|            329426|            288291|            312334|            325902|            325900|
|   mean| 267.7638233876759| 12.20040338027042| 4.754634676471857|16.478071791024288| 2.716952783646249|1.0867332176781614|
| stddev|228.76110485218726|17.658622052648546|  8.29449253521343| 21.55312033365352|139.99363784028645|55.992795617725115|
|    min|               0.0|               0.0|               0.0|               0.0|               0.0|               0.0|
|    25%|              88.0|               0.0|               0.0|              1.54|            0.0675|

### 4.4 Overview DF2:

In [41]:
# general overview of the new DF2

print ("Summary of columns:")
DF2.summary().toPandas().T

Summary of columns:


,0,1,2,3,4,5,6,7
summary,count,mean,stddev,min,25%,50%,75%,max
Creator,346468,Infinity,NaN,11vjspdpt,Infinity,Infinity,Infinity,zoneblockscommunity
Created_t,346468,1.537011007757152E9,5.0365319903437294E7,1332933734,1489074211,1489144192,1587660476,1614042745
Last_modified_t,346468,1.5829966209006488E9,2.3907735249800734E7,1421079327,1587583908,1587635759,1587664958,1614043140
Product_name,340995,6.63058998982E10,1.8445598471286032E11,"!, Chocolate Bear Crunch Popcorn",180.0,479.0,7.2392348312E10,울무차
Countries_en,346468,None,None,"Albania,Algeria,Andorra,Argentina,Australia,Au...",None,None,None,Virgin Islands of the United States
Traces_en,3825,None,None,"1-milkfat,Vitamins-a-d-added",None,None,None,ro:apa
Additives_tags,209009,None,None,en:e100,None,None,None,en:e999
Main_category_en,292653,135.63636363636363,101.34742787246076,0,70.0,120.0,190.0,调味品
Image_url,35619,None,None,https://static.openfoodfacts.org/images/produc...,None,None,None,https://static.openfoodfacts.org/images/produc...


In [42]:
analysis_col_list = DF2.schema.names

print("Checking for null values in columns:")
DF2.select([count(when(col(c).isNull(), c)).alias(c) for c in analysis_col_list]).toPandas()

Checking for null values in columns:


,Creator,Created_datetime,Created_t,Last_modified_datetime,Last_modified_t,Product_name,Countries_en,Traces_en,Additives_tags,Main_category_en,...,Packaging_tags,Categories_en,Ingredients_text,Additives_en,Energy-kcal_100g,Fat_100g,Saturated-fat_100g,Sugars_100g,Salt_100g,Sodium_100g
0,0,0,0,0,0,5473,0,342643,137459,53815,...,339649,53815,24765,137459,17751,17042,58177,34134,20566,20568


In [16]:
print("Checking for distinct values in columns:")
DF2.select([countDistinct(c).alias(c) for c in analysis_col_list]).toPandas().T

Checking for distinct values in columns:


,0
Creator,763
Created_datetime,116458
Created_t,116458
Last_modified_datetime,130616
Last_modified_t,130616
Product_name,236056
Countries_en,388
Traces_en,551
Additives_tags,47935
Main_category_en,2774


## 5. Business question
** Takeaways: We always add "isNotNull()", since we do not want null values**

### 5.1 Find the oldest product.

In [17]:
print("The oldest product in the system")
min_time = DF2.select(min('created_datetime')).first()[0]
oldest_product = (
    DF2
    .where(DF2["created_datetime"] == min_time)
    .where(DF2["product_name"].isNotNull())
    .select("created_datetime", "product_name")
    .show())

The oldest product in the system
+-------------------+---------------+
|   created_datetime|   product_name|
+-------------------+---------------+
|2012-03-28 13:22:14|Bière trappiste|
+-------------------+---------------+



### 5.2 Find the newest product.

In [18]:
print("The newest product in the system")
max_time = DF2.select(max('created_datetime')).first()[0]
oldest_product = (
    DF2
    .where(DF2["created_datetime"] == max_time)
    .where(DF2["product_name"].isNotNull())
    .select("created_datetime", "product_name")
    .show())

The newest product in the system
+-------------------+-------------+
|   created_datetime| product_name|
+-------------------+-------------+
|2021-02-23 02:12:25|Coconut water|
+-------------------+-------------+



### 5.3 Average product age, where age means how long the product has been in the system.

In [19]:
from pyspark.sql.functions import current_date,from_utc_timestamp

average_product_age = DF2.withColumn('age', ((unix_timestamp(current_date()))-col('created_t'))/(86400*365))\
   .select(avg('age')).first()[0]
print("THE AVERAGE PRODUCT AGE IS ",average_product_age,"YEARS")


print("\nTop 5 oldest prodcuts by average age:")
DF2.withColumn('age', ((unix_timestamp(current_date()))-col('created_t'))/(86400*365))\
   .groupBy('product_name')\
   .agg(avg('age')).orderBy(col('avg(age)').desc()).limit(5).show()

print("\nTop 5 newest prodcuts by average age:")
DF2.withColumn('age', ((unix_timestamp(current_date()))-col('created_t'))/(86400*365))\
   .groupBy('product_name')\
   .agg(avg('age')).orderBy(col('avg(age)').asc()).limit(5).show()

THE AVERAGE PRODUCT AGE IS  2.475285142150168 YEARS

Top 5 oldest prodcuts by average age:
+--------------------+-----------------+
|        product_name|         avg(age)|
+--------------------+-----------------+
|     Bière trappiste|8.946533041603248|
|Dove Anti-Perspirant|8.722048833079654|
|    Mini Chips Ahoy!| 8.71850814941654|
|Aloe vera drink w...|8.716527428970066|
|Classic energy drink|8.707039732369356|
+--------------------+-----------------+


Top 5 newest prodcuts by average age:
+--------------------+--------------------+
|        product_name|            avg(age)|
+--------------------+--------------------+
|Rosemary garlic c...|  0.0326263635210553|
|Purple sweet pota...| 0.03263092973110096|
|     Spinach gnocchi|0.032634164129883306|
|     Trail mix snack| 0.03266321029934044|
|Mexican Chocolate...| 0.03266695205479452|
+--------------------+--------------------+



### 5.4 List of other countries where products are sold too.

In [20]:
List_of_countries = (
    DF2
    .select(explode(split(col("countries_en"), ',')).alias('List of Countries'))
    .distinct()
    .filter(col("countries_en") != 'US') # We dont include ourselves(US) since we want to find "other countries"
) 
print("Products were sold in", len(List_of_countries.toPandas()), "other countries as follows:")
List_of_countries.toPandas()

Products were sold in 267 other countries as follows:


,List of Countries
0,Côte d'Ivoire
1,Luxemburgo
2,Middle-east-africa
3,Anguilla
4,Czech-republic-čeština
...,...
262,Vietnam
263,Indonesia-bahasa-indonesia
264,Mali
265,Netherlands


### 5.5 Identify category of products and the compute:

   #### 5.5.1 Number of products by category



In [21]:
product_count_by_category = DF2.withColumn("category", explode(split(col("categories_en"), ",")))\
   .groupBy("category")\
   .count()\
   .orderBy(col("count").desc())
#total_products = product_count_by_category.agg(sum('count')).first()[0]
total_products = DF2.count()
product_count_by_category.withColumn('percent of all products (%)', round(100*col('count')/total_products,2)).show()

+--------------------+-----+---------------------------+
|            category|count|percent of all products (%)|
+--------------------+-----+---------------------------+
|              Snacks|70947|                      20.48|
|Plant-based foods...|69303|                       20.0|
|   Plant-based foods|59341|                      17.13|
|        Sweet snacks|33837|                       9.77|
|           Groceries|30711|                       8.86|
|           Beverages|28955|                       8.36|
|Cereals and potatoes|24693|                       7.13|
|  Biscuits and cakes|24018|                       6.93|
|              Sauces|23348|                       6.74|
|        Frozen foods|22786|                       6.58|
|             Dairies|22558|                       6.51|
|     Confectioneries|22175|                        6.4|
|Fruits and vegeta...|19157|                       5.53|
|     Fermented foods|18094|                       5.22|
|Fermented milk pr...|18088|   

   #### 5.5.2 List containing names of products by category 

In [22]:
DF2.withColumn("product",col("product_name"))\
   .withColumn("category", col('categories_en'))\
   .groupBy("product","category")\
   .count()["product","category"]\
   .where(col('product').isNotNull()&col('category').isNotNull())\
   .orderBy('product','category').toPandas()

,product,category
0,"!Ajua!, Caffeine Free Soda, Mandarin Orange","Beverages,Carbonated drinks,Sodas"
1,"!Ajua!, Caffeine Free Soda, Pineapple","Beverages,Carbonated drinks,Sodas"
2,"!Ajua!, Caffeine Free Soda, Tutti Fruitti Frui...","Beverages,Carbonated drinks,Sodas"
3,"!Holla Nolla!, Bon Temps!, Real Restaurant Tor...","Snacks,Salty snacks,Appetizers,Chips and fries..."
4,""" a late one"" soft drink","Beverages,Carbonated drinks,Sodas"
...,...,...
208244,تركيا,ar:092222
208245,‘Nana Pops Banana-Rama Supersnacks,Snacks
208246,オロナミンCドリンク,"Beverages,Carbonated drinks,Unsweetened bevera..."
208247,果の每日茶,Beverages


### 5.6 Identify traces and compute:

   #### 5.6.1 Number of products by trace
    

In [23]:
DF2.withColumn("traces", explode(split(col("traces_en"), ",")))\
   .groupBy("traces")\
   .count()\
   .orderBy(col("count").desc())\
   .show()

+--------------------+-----+
|              traces|count|
+--------------------+-----+
|            Soybeans| 1547|
|                Milk| 1391|
|                Nuts| 1212|
|              Gluten| 1126|
|                Eggs|  620|
|             Peanuts|  527|
|        Sesame seeds|  154|
|Sulphur dioxide a...|   84|
|              Celery|   68|
|             Mustard|   64|
|                Fish|   49|
|             Coconut|   37|
|         Crustaceans|   34|
|                None|   25|
|  es:nueces-de-arbol|   13|
|                Salt|   12|
|Crustacean-shellfish|   10|
|           Shellfish|   10|
|         Citric-acid|    8|
|            Molluscs|    8|
+--------------------+-----+
only showing top 20 rows



#### 5.6.2 List containing names of products by trace

In [24]:
DF2.withColumn("trace", col('traces_en'))\
   .withColumn("product",col("product_name"))\
   .groupBy("trace","product")\
   .count()["product","trace"]\
   .where(col('product').isNotNull()&col('trace').isNotNull())\
   .orderBy('product','trace').toPandas()

,product,trace
0,100 calorie snacks snack bites,"Gluten,Soybeans"
1,100% Pure Pumpkin Canned,Chemicals-known-to-california-prop-650-to-caus...
2,100% Whole wheat spaghetti,Eggs
3,100% cocoa Unsweetened Chocolate,"Milk,Nuts,Soybeans"
4,100% whole grain bread,Milk
...,...,...
3506,white french bread,"Eggs,Soybeans"
3507,whole milk,Milk
3508,wholesome medley,"Nuts,Peanuts,Soybeans"
3509,wilton food color wilton icing color,"Fish,Milk,Molluscs,Nuts,Sesame seeds,Soybeans"


### 5.7 Data quality analysis on fields of interest (see appendix 1):

   #### 5.7.1 Number of products with complete info.
    

In [25]:
fields_of_interest = ["Creator",\
                 "Created_datetime",\
                 "Created_t",\
                 "Last_modified_datetime",\
                 "Last_modified_t",\
                 "Product_name",\
                 "Countries_en",\
                 "Traces_en",\
                 "Additives_tags",\
                 "Main_category_en",\
                 "Image_url",\
                 "Quantity",\
                 "Packaging_tags",\
                 "Categories_en",\
                 "Ingredients_text",\
                 "Additives_en",\
                 "Energy-kcal_100g",\
                 "Fat_100g",\
                 "Saturated-fat_100g",\
                 "Sugars_100g",\
                 "Salt_100g",\
                 "Sodium_100g"]
complete_info = DF2.select(fields_of_interest).na.drop()
count_of_complete_info = complete_info.distinct().count()
print("Number of products with complete info on fields of interest:",count_of_complete_info)

Number of products with complete info on fields of interest: 517


 #### 5.7.2 % of products without complete analysis per 100g

In [26]:
count_incomplete_analysis_100g = DF2.filter(col("Energy-kcal_100g").isNull() | \
                           col("Fat_100g").isNull() | \
                           col("Saturated-fat_100g").isNull() | \
                           col("Sugars_100g").isNull() | \
                           col("Salt_100g").isNull() | \
                           col("Sodium_100g").isNull()).count()

percentage_incomplete_analysis_100g = count_incomplete_analysis_100g/DF2.count()*100

print("% of products without complete analysis per 100g is", percentage_incomplete_analysis_100g,"%")

% of products without complete analysis per 100g is 20.272867912765395 %


In [27]:
count_no_analysis_100g = DF2.filter(col("Energy-kcal_100g").isNull() & \
                           col("Fat_100g").isNull() & \
                           col("Saturated-fat_100g").isNull() & \
                           col("Sugars_100g").isNull() & \
                           col("Salt_100g").isNull() & \
                           col("Sodium_100g").isNull()).count()

percentage_no_analysis_100g = count_no_analysis_100g/DF2.count()*100

print("% of products without any analysis per 100g is", percentage_no_analysis_100g,"%")


% of products without any analysis per 100g is 4.616587967719962 %


 #### 5.7.3 % of products without additives info

In [28]:
count_without_additive_info = DF2.filter(col('additives_en').isNull() |\
                                         col('additives_tags').isNull()).count()
count_without_additive_info

percentage_without_additive_info = count_without_additive_info/DF2.count()*100

print("Percentage of products without additive information is",percentage_without_additive_info,"%")

Percentage of products without additive information is 39.674371081889234 %


#### 5.7.4 % of products without traces info

In [29]:
count_without_traces_info = DF2.filter(col('traces_en').isNull()).count()
count_without_traces_info

percentage_without_traces_info = count_without_traces_info/DF2.count()*100

print("Percentage of products without traces information is",percentage_without_traces_info,"%")

Percentage of products without traces information is 98.89600193957307 %


### 5.8 Data profiling on fields of interest (see appendix 1):

   #### 5.8.1 Stats on analysis per 100g fields

In [30]:
DF_columnsofinterest_100g = (DF2
    .where(col("Fat_100g").isNotNull())
    .where(col("Saturated-fat_100g").isNotNull())
    .where(col("Sugars_100g").isNotNull())
    .where(col("Salt_100g").isNotNull())
    .where(col("Sodium_100g").isNotNull())
    .withColumn("Level_of_total_fat_per_100g", 
                when(col("Fat_100g")<=3, "Low")
                .when((col("Fat_100g")>3) & (col("Fat_100g")<=17.5), "Normal")
                .otherwise("High")
               )
    .withColumn("Level_of_saturated_fat_per_100g", 
                when(col("Saturated-fat_100g")<=1.5, "Low")
                .when((col("Saturated-fat_100g")>1.5) & (col("Saturated-fat_100g")<=5.0), "Normal")
                .otherwise("High")
               )
    .withColumn("Level_of_sugar_per_100g", 
                when(col("Sugars_100g")<=5, "Low")
                .when((col("Sugars_100g")>5) & (col("Sugars_100g")<=22.5), "Normal")
                .otherwise("High")
               )
    .withColumn("Level_of_sodium_per_100g", 
                when(col("Sodium_100g")<=0.3, "Low")
                .when((col("Sodium_100g")>0.3) & (col("Sodium_100g")<=1.5), "Normal")
                .otherwise("High")
               )
    .withColumn("Level_of_salt_per_100g", 
                when(col("Salt_100g")<=0.3, "Low")
                .when((col("Salt_100g")>0.3) & (col("Salt_100g")<=1.5), "Normal")
                .otherwise("High")
               )
    .select("Product_name", "Level_of_total_fat_per_100g", "Level_of_saturated_fat_per_100g", \
            "Level_of_sugar_per_100g", "Level_of_sodium_per_100g", "Level_of_salt_per_100g")
)

def perc_prod_100g (component):
    component_level = 'Level_of_'+component+'_per_100g'
    count_prod_high_component = DF_columnsofinterest_100g.where(col(component_level) == "High")\
                                                   .count()
    count_prod_norm_component = DF_columnsofinterest_100g.where(col(component_level) == "Normal")\
                                                   .count()
    count_prod_low_component = DF_columnsofinterest_100g.where(col(component_level) == "Low")\
                                                   .count()
    perc1 = 100*count_prod_high_component/DF_columnsofinterest_100g.count()
    perc2 = 100*count_prod_norm_component/DF_columnsofinterest_100g.count()
    perc3 = 100*count_prod_low_component/DF_columnsofinterest_100g.count()
    print(perc1,"% products have","high",component)
    print(perc2,"% products have","normal",component)
    print(perc3,"% products have","low",component,"\n")

perc_prod_100g('total_fat')
perc_prod_100g('saturated_fat')
perc_prod_100g('sugar')
perc_prod_100g('sodium')
perc_prod_100g('salt')

30.270846123289154 % products have high total_fat
32.22924415875194 % products have normal total_fat
37.49990971795891 % products have low total_fat 

31.5983532555704 % products have high saturated_fat
19.944747390849013 % products have normal saturated_fat
48.45689935358059 % products have low saturated_fat 

26.299880827705753 % products have high sugar
25.033765483370047 % products have normal sugar
48.6663536889242 % products have low sugar 

4.024773392076848 % products have high sodium
43.99443862626846 % products have normal sodium
51.980787981654686 % products have low sodium 

22.72832328193276 % products have high salt
42.050124589216715 % products have normal salt
35.221552128850526 % products have low salt 



## 6. Food color analysis

In [124]:
foodColor1 = r'E1[0-9]{2} -'
foodColor2 = r'E1[0-9]{2}[a-z] -'
colors = DF2.withColumn("colors", explode(split(col("additives_en"), ",")))\
   .withColumn("product_name",col("product_name"))\
   .filter(col('colors').rlike(foodColor1)|col('colors').rlike(foodColor2))\
   .orderBy(col('product_name').desc())\
   .select('product_name','colors')

In [125]:
prod_no_colors = colors.where(col('product_name').isNotNull()).groupBy('product_name')\
      .agg(countDistinct('colors').alias('number of colors'))\
      .orderBy(col('number of colors').desc()).cache()
print("List of Products with number of colors:")
prod_no_colors.show()

List of Products with number of colors:
+--------------------+----------------+
|        product_name|number of colors|
+--------------------+----------------+
|           Ice cream|              16|
|   Premium ice cream|              16|
|             Cookies|              15|
|              Cereal|              13|
|Frosted Sugar Coo...|              13|
|Frosted sugar coo...|              13|
|           Trail mix|              13|
|   Premium Ice Cream|              13|
|            Cupcakes|              12|
|          Cheesecake|              12|
|    Sandwich cookies|              12|
|              Danish|              12|
|               Candy|              12|
|           Ice Cream|              12|
|        Granola bars|              11|
|       Sugar cookies|              11|
|     Light ice cream|              11|
|            Brownies|              11|
|         Onion rings|              11|
|              Gelato|              11|
+--------------------+----------------+


In [126]:
percentage_prod_colored = 100*colors.groupBy('product_name')\
                                    .agg(count('colors')).count()/DF2.count()
print(int(percentage_prod_colored),"% of all food products (",\
      colors.groupBy('product_name').agg(count('colors')).count(),\
      "products ) sold in America contain food color")

15 % of all food products ( 55001 products ) sold in America contain food color


In [127]:
avg_no_colors = int(prod_no_colors.agg(avg(col('number of colors'))).first()[0])
print("In America, on average, at least", avg_no_colors,"food color additives \
are used in food products that are artificially color")


In America, on average, at least 2 food color additives are used in food products that are artificially color


In [137]:
no_of_distinct_colors = colors.agg(countDistinct(col('colors'))).first()[0]
print("A total of",no_of_distinct_colors,"different food color additives are used in food products sold in America")

colors.select(col('colors')).distinct().orderBy('colors').show(58, truncate = False)

A total of 58 different food color additives are used in food products sold in America
+----------------------------------------------------------+
|colors                                                    |
+----------------------------------------------------------+
|E100 - Curcumin                                           |
|E101 - Riboflavin                                         |
|E101i - Riboflavin                                        |
|E102 - Tartrazine                                         |
|E103 - Alkannin                                           |
|E104 - Quinoline yellow                                   |
|E105 - E105 food additive                                 |
|E106 - Flavin mononucleotide                              |
|E107 - Yellow 2G                                          |
|E110 - Sunset yellow FCF                                  |
|E111 - Orange GGN                                         |
|E120 - Cochineal                                          

In [91]:
prod_w_hyper_color = colors.where("colors like 'E110%'" or\
                                  "colors like 'E104%'" or\
                                  "colors like 'E122%'" or\
                                  "colors like 'E129%'" or\
                                  "colors like 'E102%'" or\
                                  "colors like 'E124%'")

print(prod_w_hyper_color.count(),"products (",\
      int(100*prod_w_hyper_color.count()/DF2.count()),\
      "% of all products ) sold in America contain food color additives that cause hyperactivity in kids")

#Source: https://www.food.gov.uk/safety-hygiene/food-additives

14989 products ( 4 % of all products ) sold in America contain food color additives that cause hyperactivity in kids


In [136]:
foodColorList = [E100,E101,E102,E104,E110,E120,E122,E123,E124,E127,E128,E129,E131,E132,E133,E140i,E140ii,E141i,E141ii,E142,E150,E151,E153,E154,E155,E160a,E160b,E160c,E160d,E160e,E160f,E161b,E161g,E162,E163,E171,E172,E173,E174,E175,E180]

NameError: name 'E100' is not defined